### Create a notebook for Cash Karp integration of coupled ODEs


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define our ODEs

In [ ]:
def dydx(x,f):
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros_like(f)
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx = -y
    y_derivs[1] = -1*y
    
    #here we have to return an array
    return y_derivs
    

### Cash-Karp core method

In [ ]:
def cash_Karp_mv_core(dydx,xi,yi,nv,h):
    
    ni = 7
    nj= 6
    ci = np.zeros(7)
    aij = np.zeros((7,6))
    bi = np.zeros(7)
    bis = np.zeros(7)
    
    #initialize the c's
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    #initialize a's
    aij[2,1] = 1.0/5.0
    aij[3,1] = 3.0/40.0
    aij[4,1] = 3.0/10.0
    aij[5,1] = -11.0/54.0
    aij[6,1] = 1631./55296.
    
    aij[3,2] = 9.0/40.0
    aij[4,2] = -9.0/10.0
    aij[5,2] = 5.0/2.0
    aij[6,2] = 175./512.
    
    aij[4,3] = 6.0/5.0
    aij[5,3] = -70.0/27.0
    aij[6,3] = 575./13824.
    
    aij[5,4] = -35.0/27.0
    aij[6,4] = 44275./110592.
    
    aij[6,5] = 253./4096.
    
    #initialize b's
    bi[1] = 37./378.0
    bi[2] = 0.
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.0
    bi[6] = 512./1771.
    
    #initialize bis
    bis[1] = 2825./27648.
    bis[2] = 0.0
    bis[3] = 18575./48384.
    bis[4] = 13525./55296.
    bis[5] = 277.0/14336.0
    bis[6] = 1.0/4.0
    
    #declare k as 2D array
    ki = np.zeros((ni,nv))
    
    #compute ki
    for i in range(1,ni):
        xn = xi + ci[i]*h
        
        yn = yi.copy()
        for j in range(1,i):
            yn += aij[i,j] * ki[j,:]
        
        #get k
        ki[i,:] = h * dydx(xn,yn)
        #print('check')
    #get ynpo and ynpo*
    ynpo = yi.copy()
    ynpos = yi.copy()
    
    for i in range(1,ni):
        ynpo += bi[i] * ki[i,:]
        ynpos += bis[i] * ki[i,:]
      
    
    #get error
    print(ynpo, ynpos)
    delta = np.fabs(ynpo-ynpos)
    #print("inside delta", delta,ki[:,0],ynpo,ynpos)
    
    return ynpo, delta
        


### Define adaptive step size driver for Cash-Karp

In [ ]:
def cash_karp_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set max iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the current step size
    h_step = h
    
    #adjust step if necessary 
    while(Delta.max()/tol>1.0):
        #get y-new and err estimate
        y_ipo, delta = cash_Karp_mv_core(dydx,x_i,y_i,nv,h)
        
             
        #if the error is too large take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("Too many iterations in cash_karp_mk_ad()")
            raise StopIteration("Ending after i = ",i)
        
        #iterate
        i+=1

        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step*(Delta.max()/tol)**(-0.9),h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we took
    return y_ipo, h_new, h_step

    

### Define a Wrapper for Cash Karp method

In [ ]:
def cash_karp_mv(dydx,a,b,y_a,tol):
    
    #dfdx is the derivative with respect to x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    #set the number of coupled odes to the 
    #size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y+1
        yi_po, h_new, h_step = cash_karp_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_po, h_new, h_step = cash_karp_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi = y_ipo.copy()
        
        #add the step to the arrays
        x = np.append(x,xi)
        yi_po = np.zeros((len(x),nv))
        yi_po[0:len(x)-1,:] = y[:]
        yi_po[-1,:] = yi[:]
        del y
        y = yi_po
        
                         
        #prevent too many iterations
        if(i>=imax):
            print("Maxinmum iterations reached.")
            raise STOPITERATION("Iterations number = ", i)
        
        #iterate
        i += 1
        #output some info 
                        
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)                   
        print(s)
    
        #break if new xi is == b                     
        if(xi==b):
            flag = 0
                         
    #return the answer
    return x,y
    
    

### Perform the integration

In [ ]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = cash_karp_mv(dydx,a,b,y_0,tolerance)


In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

# plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)